In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
count =1
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["release_year"])
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_l4,Approved,KEGG,Ingredient,date,release_year,成分,variable,id,revenue
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,3.261297e+08
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,2.377048e+08
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,6.583945e+08
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,6.363609e+08
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,3.368936e+08


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [3]:
data["elapsed"]=data["year"]-data["release_year"]
#　不要なところを除外
# data.loc[(data['elapsed'] > 10) | (data["elapsed"]< -5), 'elapsed'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elapsed"],prefix="elapsed")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","id","後発品区分"])[sumcolumns].sum().reset_index()
# save temporarily
colnames=dummies.columns.to_list()
if count:
    # count ver.
    aggregated_df = pd.concat([data.groupby(['year', 'id',"後発品区分"]).first(), dummies.groupby([data['year'], data['id'],data["後発品区分"]]).sum()], axis=1).reset_index()
else:
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"後発品区分"]).first(), 
        dummies.groupby([data['year'], data['id'],data["後発品区分"]]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
# sumをとったところを追加する
aggregated_df[sumcolumns]=sums[sumcolumns]
aggregated_df
aggregated_df.head()

,year,id,後発品区分,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,総計,...,elapsed_22,elapsed_23,elapsed_24,elapsed_25,elapsed_26,elapsed_27,elapsed_28,elapsed_29,elapsed_30,elapsed_31
0,2014,1147.0,0.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,1.525181e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,1147.0,1.0,114.0,解熱鎮痛消炎剤,661140205.0,ボナフェック坐剤５０ ５０ｍｇ,1147700J3017,19.3,1.271671e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,1149.0,0.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.5,1.233739e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,1149.0,1.0,114.0,解熱鎮痛消炎剤,620099301.0,ロキソプロフェンＮａ錠６０ｍｇ「サワイ」,1149019F1587,7.8,3.986192e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014,1219.0,0.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.7,4.539382e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


connect generic usage

In [4]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [5]:
# generic from ndb
dgeneric=pd.read_csv("../merged/len4_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric

,id,year,generic_share_q,generic_share_r
0,1147.0,2014,0.076961,0.059379
33,1149.0,2014,0.244198,0.053661
80,1219.0,2014,0.062114,0.053451
96,1231.0,2014,0.129137,0.064243
147,1319.0,2014,0.136869,0.080471
...,...,...,...,...
25142,4413.0,2021,0.342768,0.263456
25182,4490.0,2021,0.608051,0.363450
25386,6250.0,2021,0.502607,0.052158
25628,6290.0,2021,0.284349,0.073842


In [6]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["id","year"])
merged_df

,year,id,後発品区分,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,総計,...,elapsed_26,elapsed_27,elapsed_28,elapsed_29,elapsed_30,elapsed_31,Year,Percentage,generic_share_q,generic_share_r
0,2014,1147.0,0.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.10,1.525181e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,51.55,0.076961,0.059379
1,2014,1147.0,1.0,114.0,解熱鎮痛消炎剤,661140205.0,ボナフェック坐剤５０ ５０ｍｇ,1147700J3017,19.30,1.271671e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,51.55,0.076961,0.059379
2,2014,1149.0,0.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.50,1.233739e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,51.55,0.244198,0.053661
3,2014,1149.0,1.0,114.0,解熱鎮痛消炎剤,620099301.0,ロキソプロフェンＮａ錠６０ｍｇ「サワイ」,1149019F1587,7.80,3.986192e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,51.55,0.244198,0.053661
4,2014,1219.0,0.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.70,4.539382e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,51.55,0.062114,0.053451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,2021,6250.0,1.0,625.0,抗ウイルス剤,622487501.0,バラシクロビル錠５００ｍｇ「アスペン」,6250019F1330,112.40,1.446454e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,79.00,0.502607,0.052158
493,2021,6290.0,0.0,629.0,その他の化学療法剤,621148801.0,ダイフェン配合錠,6290100F2018,15.90,2.328826e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,79.00,0.284349,0.073842
494,2021,6290.0,1.0,629.0,その他の化学療法剤,622632901.0,イトラコナゾール内用液１％「ファイザー」,6290004S1036,32.30,9.253091e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,79.00,0.284349,0.073842
495,2021,7990.0,0.0,799.0,他に分類されない治療を主目的としない医薬品,621201002.0,Ｄソルビトール経口液７５％「コーワ」,7990001S3011,1.02,4.594945e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,79.00,0.007953,0.028184


In [7]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,後発品区分,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,総計,...,id_2659.0,id_3327.0,id_3399.0,id_3992.0,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0
0,2014,1147.0,0.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.10,1.525181e+08,...,0,0,0,0,0,0,0,0,0,0
1,2014,1147.0,1.0,114.0,解熱鎮痛消炎剤,661140205.0,ボナフェック坐剤５０ ５０ｍｇ,1147700J3017,19.30,1.271671e+07,...,0,0,0,0,0,0,0,0,0,0
2,2014,1149.0,0.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.50,1.233739e+10,...,0,0,0,0,0,0,0,0,0,0
3,2014,1149.0,1.0,114.0,解熱鎮痛消炎剤,620099301.0,ロキソプロフェンＮａ錠６０ｍｇ「サワイ」,1149019F1587,7.80,3.986192e+09,...,0,0,0,0,0,0,0,0,0,0
4,2014,1219.0,0.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.70,4.539382e+07,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,2021,6250.0,1.0,625.0,抗ウイルス剤,622487501.0,バラシクロビル錠５００ｍｇ「アスペン」,6250019F1330,112.40,1.446454e+08,...,0,0,0,0,0,0,0,1,0,0
493,2021,6290.0,0.0,629.0,その他の化学療法剤,621148801.0,ダイフェン配合錠,6290100F2018,15.90,2.328826e+08,...,0,0,0,0,0,0,0,0,1,0
494,2021,6290.0,1.0,629.0,その他の化学療法剤,622632901.0,イトラコナゾール内用液１％「ファイザー」,6290004S1036,32.30,9.253091e+07,...,0,0,0,0,0,0,0,0,1,0
495,2021,7990.0,0.0,799.0,他に分類されない治療を主目的としない医薬品,621201002.0,Ｄソルビトール経口液７５％「コーワ」,7990001S3011,1.02,4.594945e+07,...,0,0,0,0,0,0,0,0,0,1


In [8]:
if not(count):
    df_with_dummies.to_csv("../merged/len4_ndb_DN.csv",encoding="shift-jis")
else:
    # count ver
    df_with_dummies.to_csv("../merged/len4_ndb_sum_DN.csv",encoding="shift-jis")